# csvのデータ作成

In [9]:
import pandas as pd
import random

# 1. マスターデータの定義（候補をさらに拡充）
items = [
    "ビール", "日本酒", "ウイスキー", "ワイン", "焼酎", "チューハイ", # 酒類
    "ポテトチップス", "チョコレート", "クッキー", "せんべい", "ナッツ", # 菓子
    "おにぎり", "サンドイッチ", "からあげ", "サラダ", "パスタ",       # 食品
    "コーヒー", "緑茶", "紅茶", "コーラ", "エナジードリンク",          # 飲料
    "シャンプー", "洗剤", "ティッシュ", "歯磨き粉", "マスク"           # 日用品
]

stores = [
    "イトーヨーカドー", "イオン", "西友", "ライフ", # スーパー
    "セブンイレブン", "ローソン", "ファミリーマート", # コンビニ
    "マツモトキヨシ", "ウエルシア" # ドラッグストア
]

# 2. データ生成設定
num_users = 50        # 作成するユーザー数
min_trips = 3         # ユーザーごとの来店回数（最小）
max_trips = 10        # ユーザーごとの来店回数（最大）

all_sequences = []

for user_id in range(1, num_users + 1):
    # 一時的なリスト
    seq_items = []
    seq_timestamps = []
    seq_stores = []
    
    # このユーザーの買い物回数
    num_trips = random.randint(min_trips, max_trips)
    
    # 時間の初期値
    current_time = random.randint(1, 10)
    
    for _ in range(num_trips):
        # --- 1回の買い物（バスケット） ---
        
        # 時間を進める
        current_time += random.randint(1, 5)
        # 店舗を決める
        current_store = random.choice(stores)
        
        # 同一バスケット内の購入数（1〜5個）
        basket_size = random.choices([1, 2, 3, 4, 5], weights=[30, 30, 20, 15, 5])[0]
        
        # アイテムを選ぶ（バスケット内では重複なしとする）
        basket_items = random.sample(items, k=basket_size)
        
        # --- シーケンスに追加 ---
        # 同じバスケットの商品は、同じtimestamp、同じstoreを持つ
        for item in basket_items:
            seq_items.append(item)
            seq_timestamps.append(current_time)
            seq_stores.append(current_store)
            
    # ユーザー1人分のデータを辞書化
    user_record = {
        "user_id": user_id,
        "item_sequence": seq_items,
        "time_sequence": seq_timestamps,
        "store_sequence": seq_stores
    }
    all_sequences.append(user_record)

# 3. DataFrame作成
df = pd.DataFrame(all_sequences)

# 4. 確認表示（ご希望の形式になっているか確認）
print("--- データプレビュー（最初の5行） ---")
print(df.head())

# データの中身の例を表示
print("\n--- 1行目のデータ詳細 ---")
print(f"User ID: {df.iloc[0]['user_id']}")
print(f"Items  : {df.iloc[0]['item_sequence']}")
print(f"Times  : {df.iloc[0]['time_sequence']}")
print(f"Stores : {df.iloc[0]['store_sequence']}")

# 5. CSV出力
csv_filename = "seq2pat_user_sequences.csv"
# リスト形式を保ったままCSVにするため、pandasのデフォルト挙動（文字列表現）を利用します
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\nCSVファイル '{csv_filename}' を保存しました。")

--- データプレビュー（最初の5行） ---
   user_id                                      item_sequence  \
0        1  [エナジードリンク, ワイン, クッキー, 日本酒, サンドイッチ, 紅茶, ビール, 日本...   
1        2  [クッキー, マスク, コーヒー, サンドイッチ, シャンプー, マスク, ウイスキー, ク...   
2        3  [マスク, コーヒー, サラダ, サンドイッチ, サラダ, エナジードリンク, おにぎり, ...   
3        4  [サンドイッチ, マスク, せんべい, チューハイ, サラダ, コーヒー, パスタ, サラダ...   
4        5  [クッキー, チューハイ, クッキー, ワイン, マスク, チョコレート, チョコレート, ...   

                                       time_sequence  \
0  [12, 12, 12, 16, 16, 16, 18, 20, 20, 25, 25, 3...   
1  [13, 13, 13, 14, 14, 15, 15, 20, 20, 20, 23, 2...   
2  [7, 7, 9, 13, 13, 13, 13, 13, 15, 18, 21, 21, ...   
3      [3, 8, 8, 12, 12, 12, 13, 13, 15, 15, 15, 17]   
4  [8, 8, 9, 9, 13, 13, 14, 14, 14, 17, 17, 20, 2...   

                                      store_sequence  
0  [ファミリーマート, ファミリーマート, ファミリーマート, ローソン, ローソン, ローソ...  
1  [イオン, イオン, イオン, ライフ, ライフ, セブンイレブン, セブンイレブン, 西友...  
2  [ローソン, ローソン, ファミリーマート, ローソン, ローソン, ローソン, ローソン,...  
3  [イトーヨーカドー, イトーヨーカドー, イトーヨ

## dfをインタラクションベースのものに変換する

In [11]:
import pandas as pd
import ast 

csv_filename = "seq2pat_user_sequences.csv"

df_loaded = pd.read_csv(
    csv_filename,
    converters={
        "item_sequence": ast.literal_eval,
        "time_sequence": ast.literal_eval,
        "store_sequence": ast.literal_eval
    }
)

df_interaction = df_loaded.explode(
    ["item_sequence", "time_sequence", "store_sequence"], 
    ignore_index=True
)

df_interaction = df_interaction.rename(columns={
    "item_sequence": "item",
    "time_sequence": "timestamp",
    "store_sequence": "store"
})

print(f"変換後のデータ行数: {len(df_interaction)}")
print(df_interaction.head(10))
# インタラクションベースのcsvを作成
df_interaction.to_csv("interaction_base_data.csv", index=False, encoding='utf-8-sig')
# itemの名前でソートする
df_sorted = df_interaction.sort_values(
    by=["user_id", "timestamp", "item"], 
    ascending=[True, True, True]
)
df_sorted.to_csv("sorted_interaction_base_data.csv", index=False, encoding='utf-8-sig')

変換後のデータ行数: 835
   user_id      item timestamp     store
0        1  エナジードリンク        12  ファミリーマート
1        1       ワイン        12  ファミリーマート
2        1      クッキー        12  ファミリーマート
3        1       日本酒        16      ローソン
4        1    サンドイッチ        16      ローソン
5        1        紅茶        16      ローソン
6        1       ビール        18        西友
7        1       日本酒        20     ウエルシア
8        1      せんべい        20     ウエルシア
9        1     ティッシュ        25       イオン


# インタラクションベースのdfをユーザーごとのリストに変換する

In [12]:
# ユーザーごとにリストにまとめる
df_final_sequences = df_sorted.groupby('user_id').agg({
    'item': list,       # ソートされた順序でリスト化されます
    'timestamp': list,  # itemに対応した順序になります
    'store': list       # itemに対応した順序になります
}).reset_index()

# カラム名を元の形式に戻す（必要であれば）
df_final_sequences = df_final_sequences.rename(columns={
    'item': 'item_sequence',
    'timestamp': 'time_sequence',
    'store': 'store_sequence'
})

print("\n--- 最終的な整形済みデータ（リスト形式） ---")
print(df_final_sequences.head())

# 確認：中身を見てみる
print("\n--- 1人目のデータ詳細 ---")
print("Items:", df_final_sequences.iloc[0]['item_sequence'])
print("Times:", df_final_sequences.iloc[0]['time_sequence'])


--- 最終的な整形済みデータ（リスト形式） ---
   user_id                                      item_sequence  \
0        1  [エナジードリンク, クッキー, ワイン, サンドイッチ, 日本酒, 紅茶, ビール, せん...   
1        2  [クッキー, コーヒー, マスク, サンドイッチ, シャンプー, ウイスキー, マスク, ク...   
2        3  [コーヒー, マスク, サラダ, おにぎり, エナジードリンク, クッキー, サラダ, サン...   
3        4  [サンドイッチ, せんべい, マスク, コーヒー, サラダ, チューハイ, サラダ, パスタ...   
4        5  [クッキー, チューハイ, クッキー, ワイン, チョコレート, マスク, チョコレート, ...   

                                       time_sequence  \
0  [12, 12, 12, 16, 16, 16, 18, 20, 20, 25, 25, 3...   
1  [13, 13, 13, 14, 14, 15, 15, 20, 20, 20, 23, 2...   
2  [7, 7, 9, 13, 13, 13, 13, 13, 15, 18, 21, 21, ...   
3      [3, 8, 8, 12, 12, 12, 13, 13, 15, 15, 15, 17]   
4  [8, 8, 9, 9, 13, 13, 14, 14, 14, 17, 17, 20, 2...   

                                      store_sequence  
0  [ファミリーマート, ファミリーマート, ファミリーマート, ローソン, ローソン, ローソ...  
1  [イオン, イオン, イオン, ライフ, ライフ, セブンイレブン, セブンイレブン, 西友...  
2  [ローソン, ローソン, ファミリーマート, ローソン, ローソン, ローソン, ローソン,...  
3  [イトーヨーカドー, イトーヨーカドー, 